In [1]:
# Import Libraries
import pandas as pd 
import matplotlib.pyplot as plt
import os
import time
import seaborn as sns
import json
import numpy as np
import apikeys
import requests
import shapefile   
from bokeh.io import output_file, show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper,
    LinearColorMapper,
    ContinuousColorMapper
)
from bokeh.palettes import Blues8, PuBu8, OrRd8
from bokeh.plotting import figure
from pprint import pprint
from datetime import datetime

# Solar Data

In [ ]:
#02 READ IN DATA 
#Read in NY solar data
path = "https://data.ny.gov/api/views/3x8r-34rs/rows.csv?accessType=DOWNLOAD"
#path = "../Resources/Solar_Installation_Data Raw.csv"

solar = pd.read_csv(path)
solar.head()

#03 DATA CLEANING
#NARROW TO RELEVANT VARIABLES
solar = solar[["Project Number", "City", "Zip Code",
               "Sector", "Electric Utility", 
               "Purchase Type", "Project Status",
               "Date Application Received", "Date Completed",
               "Project Cost","$Incentive", "Total Nameplate kW DC", 
               "Expected KWh Annual Production", "Contractor", 
               "Primary PV Module Manufacturer", "Location 1" ]]

#Identify Date variables
solar['Date Completed'] = pd.to_datetime(solar['Date Completed'] )


#Create additional variable
solar['$_per_watt'] = solar['Project Cost'] / solar['Total Nameplate kW DC'] / 1000
solar['$_per_watt_wincentives'] = (solar['Project Cost'] - solar['$Incentive'] ) / solar['Total Nameplate kW DC'] /1000

#Narrow down to projects that are completed
solar = solar.loc[solar["Project Status"] == "Complete"]

#Drop Missing Values
solar.dropna(how = 'any')

#Convert zip code to string for merge
#solar['Zip Code'] = solar['Zip Code'].to_string()
solar.head()

#04  Save File
#Output File

# Census Data

In [ ]:
#Read in Net Generation Data
url = f'http://api.eia.gov/series/?api_key={apikeys.eia_key}&series_id=EBA.NYIS-ALL.D.H'
response = requests.get(url).json()
pprint(response)

# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
census_data = c.acs5.get(("NAME", "B19013_001E", "B19301_001E",
                            "B23025_004E", "B23025_005E", "B17001_002E",
                            "B17001_003E","B17001_017E","B17001A_002E",
                            "B17001B_002E","B02001_004E","B17001D_002E",
                            "B17001E_002E","B17001G_002E","B17001I_002E",
                            "B17012_002E","B17012_003E","B17012_009E",
                            "B17012_014E","B01002_001E","B01002_002E",
                            "B01002_003E","B01003_001E","B25077_001E",
                            "B25088_002E","B15003_002E","B15003_017E",
                            "B15003_018E","B15003_021E","B15003_022E",
                            "B15003_023E","B15003_024E","B15003_025E",
                         ),
                         {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_df = pd.DataFrame(census_data)

# Column Reordering
census_df = census_df.rename(columns={
    "B19013_001E": "income",
    "B19301_001E": "income_per_capital",
    "B23025_004E": "employment_employed",
    "B23025_005E": "employment_unemployed",
    "B17001_002E": "poverty",
    "B17001_003E": "poverty_male",
    "B17001_017E": "poverty_female",
    "B17001A_002E": "poverty_white_alone",
    "B17001B_002E": "poverty_black_alone",
    "B02001_004E": "population_american_indian_alone",
    "B17001D_002E": "poverty_asian_alone",
    "B17001E_002E": "poverty_native_hawaiian_alone",
    "B17001G_002E": "poverty_two_or_more_races",
    "B17001I_002E": "poverty_hispanic_origin",
    "B17012_002E": "poverty_family",
    "B17012_003E": "poverty_family_married",
    "B17012_009E": "poverty_family_single_male",
    "B17012_014E": "poverty_family_single_female",
    "B01002_001E": "age",
    "B01002_002E": "median_male_age",
    "B01002_003E": "median_female_age",
    "B01003_001E": "population",
    "B25077_001E": "median_home_value",
    "B25088_002E": "median_monthly_owner_costs",
    "B15003_002E": "education_none",
    "B15003_017E": "education_high_school",
    "B15003_018E": "education_ged",
    "B15003_021E": "education_associates",
    "B15003_022E": "education_bachelors",
    "B15003_023E": "education_masters",
    "B15003_024E": "education_professional",
    "B15003_025E": "education_doctorate",
    "zip code tabulation area": "Zipcode"
})

# Add in Poverty Rate (Poverty Count / Population)
#census_pd["Poverty Rate"] = 100 * census_pd["Poverty Count"] / census_pd["Population"]

# Final DataFrame
census_df = census_df[["income", "income_per_capital", 
                       "employment_employed","employment_unemployed",
                       "poverty", "poverty_male",
                       "poverty_female","poverty_white_alone",
                       "poverty_black_alone","population_american_indian_alone",
                       "poverty_asian_alone","poverty_native_hawaiian_alone",
                       "poverty_two_or_more_races","poverty_hispanic_origin",
                       "poverty_family","poverty_family_married",
                       "poverty_family_single_male","poverty_family_single_female",
                       "age","median_male_age",
                       "median_female_age","population",
                       "median_home_value","median_monthly_owner_costs",
                       "education_none","education_high_school",
                       "education_ged","education_associates",
                       "education_bachelors","education_masters",
                       "education_professional","education_doctorate",
                       "Zipcode"
                    ]]

# Visualize
print(len(census_df))
census_df.head()

#Write to file
census_df.to_csv("census_raw.csv", encoding="utf-8", index=False)

# Merge Data Frames - Establish Analysis Data Frame

In [ ]:
# LOCATE/ READ FILES
file = 'Solar_installation_data.csv'
file1 = 'census_raw.csv'

solar_df = pd.read_csv(file)
census_df = pd.read_csv(file1)
# solar.head()
# census.head()

# Change Zipcode column name in census_df to match Zip Code column name in solar_df
census_df = census_df.rename(columns={'Zipcode':'Zip Code'})

# Merge solar_df and census_df
df = solar_df.merge(census_df, on = 'Zip Code', how='left')
df = df.drop(columns=['Unnamed: 0'])
df

#Convert Date variables to datetime
df['Date Completed'] = pd.to_datetime(df['Date Completed'])
df['Date Application Received'] = pd.to_datetime(df['Date Application Received'])

#Convert datetimes to unix timestamps
def convert_time(time_dt):
    return int(time.mktime(time_dt.timetuple()))

#print columns with unix ts and length of project in days
df['Date Application Received Int'] = df.apply(lambda row: convert_time(row['Date Application Received']), axis=1)
df['Date Completed Int'] = df.apply(lambda row: convert_time(row['Date Completed']), axis=1)
df['Project Days Int'] = df.apply(lambda row: int((row['Date Completed Int']-row['Date Application Received Int'])/86400), axis=1)

#parse the datetime objects
df['Completed Year'] = df.apply(lambda row: row['Date Completed'].year, axis=1)
df['Completed Month'] = df.apply(lambda row: row['Date Completed'].month, axis=1)

#Create additional variables
df['$_per_watt'] = round(df['Project Cost'] / df['Total Nameplate kW DC'] / 1000, 2)
df['$_per_watt_wincentives'] = round((df['Project Cost'] - df['$Incentive'] ) / df['Total Nameplate kW DC'] /1000, 2)



df.head()

# Penetration by Sector (Residential vs. Commercial)

In [ ]:
# Read Data
p_data = "../Resources/project_data_final.csv"
p_df = pd.read_csv(p_data)
p_df['Date Completed'] = pd.to_datetime(p_df['Date Completed'])
#p_df = p_df.sort_values('Date Application Received', ascending=True)

# Drop extra column
p_df = p_df.drop(columns=['Unnamed: 0'])
#p_df.head()

# Create Year-Month Column
dates = p_df['Date Completed']
y_m = []
for date in dates:
    y_m.append(date.strftime('%Y-%m'))
    
p_df['Year-Month'] = y_m

#create residential analysis data frames
df = p_df[['Project Number','Sector','Date Completed','Expected KWh Annual Production', 'Year-Month']]

residential_df = df.loc[df['Sector'] == 'Residential']
residential_df = residential_df.reset_index()
residential_df = residential_df.drop(columns=['index'])
residential_df = residential_df.sort_values('Date Completed', ascending=True)

non_residential_df = df.loc[df['Sector'] == 'Non-Residential']
non_residential_df = non_residential_df.reset_index()
non_residential_df = non_residential_df.drop(columns=['index'])
non_residential_df = non_residential_df.sort_values('Date Completed', ascending=True)

residential_df.head()


# RESIDENTIAL APPLICATION DATE
res_df = residential_df.groupby('Year-Month')
res_date_df = res_df['Date Completed'].max()
res_date_df = res_date_df.to_frame()
res_date_df = res_date_df.sort_values('Date Completed', ascending=True)
res_date_df.head()

res_kwh_df = res_df['Expected KWh Annual Production'].sum()
res_kwh_df = res_kwh_df.to_frame()
res_kwh_df.head()

res_count_df = residential_df['Year-Month'].value_counts()
res_count_df = res_count_df.to_frame()
res_count_df = res_count_df.reset_index()
res_count_df = res_count_df.rename(columns={'Year-Month':'Project Count', 'index':'Year-Month'})
res_count_df = res_count_df.set_index('Year-Month')
res_count_df.head()

#FINAL RESIDENTIAL DATAFRAME
merge1 = res_date_df.merge(res_count_df, left_index=True, right_index=True)
final_res_df = merge1.merge(res_kwh_df, left_index=True, right_index=True)
final_res_df.head()

# NON-RESIDENTIAL APPLICATION DATE
non_res_df = non_residential_df.groupby('Year-Month')
non_res_date_df = non_res_df['Date Completed'].max()
non_res_date_df = non_res_date_df.to_frame()
non_res_date_df = non_res_date_df.sort_values('Date Completed', ascending=True)
non_res_date_df.head()

non_res_kwh_df = non_res_df['Expected KWh Annual Production'].sum()
non_res_kwh_df = non_res_kwh_df.to_frame()
non_res_kwh_df.head()

non_res_count_df = non_residential_df['Year-Month'].value_counts()
non_res_count_df = non_res_count_df.to_frame()
non_res_count_df = non_res_count_df.reset_index()
non_res_count_df = non_res_count_df.rename(columns={'Year-Month':'Project Count', 'index':'Year-Month'})
non_res_count_df = non_res_count_df.set_index('Year-Month')
non_res_count_df.head()

#FINAL NONRESIDENTIAL DATAFRAME
merge2 = non_res_date_df.merge(non_res_count_df, left_index=True, right_index=True)
final_non_res_df = merge2.merge(non_res_kwh_df, left_index=True, right_index=True)
final_non_res_df.head()


# Purchase Type Time Series

In [ ]:
# READ DATA
p_data = "../Resources/project_data_final.csv"
p_df = pd.read_csv(p_data)

# CONVERT DATE FIELD
p_df['Date Completed'] = pd.to_datetime(p_df['Date Completed'])
#p_df = p_df.sort_values('Date Application Received', ascending=True)

# DROP EXTRA COLUMN
p_df = p_df.drop(columns=['Unnamed: 0'])
p_df = p_df.dropna()

print(p_df['Purchase Type'].unique())
p_df.head(3)

# CREATING YEAR-MONTH 
dates = p_df['Date Completed']
y_m = []
for date in dates:
    y_m.append(date.strftime('%Y-%m'))
    
p_df['Year-Month'] = y_m

#CREATE ANALYSIS DATA FRAME
df = p_df[['Project Number','Sector','Date Completed','Purchase Type', 'Year-Month']]

purchase_df = df.loc[df['Purchase Type'] == 'Purchase']
purchase_df = purchase_df.reset_index()
purchase_df = purchase_df.drop(columns=['index'])
purchase_df = purchase_df.sort_values('Date Completed', ascending=True)
purchase_df = purchase_df.reset_index()
purchase_df = purchase_df.drop(columns=['index'])

lease_df = df.loc[df['Purchase Type'] == 'Lease']
lease_df = lease_df.reset_index()
lease_df = lease_df.drop(columns=['index'])
lease_df = lease_df.sort_values('Date Completed', ascending=True)
lease_df = lease_df.reset_index()
lease_df = lease_df.drop(columns=['index'])

ppa_df = df.loc[df['Purchase Type'] == 'Power Purchase Agreement']
ppa_df = ppa_df.reset_index()
ppa_df = ppa_df.drop(columns=['index'])
ppa_df = ppa_df.sort_values('Date Completed', ascending=True)
ppa_df = ppa_df.reset_index()
ppa_df = ppa_df.drop(columns=['index'])

## PURCHASE DATA
purchase_date_df = purchase_df.groupby('Year-Month')
purchase_date_df = purchase_date_df['Date Completed'].max()
purchase_date_df = purchase_date_df.to_frame()
purchase_date_df = purchase_date_df.sort_values('Date Completed', ascending=True)
purchase_date_df.head()

purchase_count_df = purchase_df['Year-Month'].value_counts()
purchase_count_df = purchase_count_df.to_frame()
purchase_count_df = purchase_count_df.reset_index()
purchase_count_df = purchase_count_df.rename(columns={'index':'Year-Month', 'Year-Month':'Project Count'})
purchase_count_df = purchase_count_df.set_index('Year-Month')
purchase_count_df.head()

# Final DataFrame for purchase_type == purchase
final_purchase_df = purchase_date_df.merge(purchase_count_df, left_index=True, right_index=True)
final_purchase_df.head()

## LEASE DATA
lease_date_df = lease_df.groupby('Year-Month')
lease_date_df = lease_date_df['Date Completed'].max()
lease_date_df = lease_date_df.to_frame()
lease_date_df = lease_date_df.sort_values('Date Completed', ascending=True)
lease_date_df.head()

lease_count_df = lease_df['Year-Month'].value_counts()
lease_count_df = lease_count_df.to_frame()
lease_count_df = lease_count_df.reset_index()
lease_count_df = lease_count_df.rename(columns={'index':'Year-Month', 'Year-Month':'Project Count'})
lease_count_df = lease_count_df.set_index('Year-Month')
lease_count_df.head()

# Final DataFrame for purchase_type == lease
final_lease_df = lease_date_df.merge(lease_count_df, left_index=True, right_index=True)
final_lease_df.head()

## POWER PURCHASE AGREEMENT
ppa_date_df = ppa_df.groupby('Year-Month')
ppa_date_df = ppa_date_df['Date Completed'].max()
ppa_date_df = ppa_date_df.to_frame()
ppa_date_df = ppa_date_df.sort_values('Date Completed', ascending=True)
ppa_date_df.head()

ppa_count_df = ppa_df['Year-Month'].value_counts()
ppa_count_df = ppa_count_df.to_frame()
ppa_count_df = ppa_count_df.reset_index()
ppa_count_df = ppa_count_df.rename(columns={'index':'Year-Month', 'Year-Month':'Project Count'})
ppa_count_df = ppa_count_df.set_index('Year-Month')
ppa_count_df.head()

# Final DataFrame for purchase_type == lease
final_ppa_df = ppa_date_df.merge(ppa_count_df, left_index=True, right_index=True)
final_ppa_df.head()


# Income Distribution

In [ ]:
# READ DATA AND DO BASIC CLEAN UP
p_data = "project_data_final.csv"
p_df = pd.read_csv(p_data)
p_df['Date Completed'] = pd.to_datetime(p_df['Date Completed'])
p_df = p_df.drop(columns=['Unnamed: 0'])
p_df.head()

#create analysis data frames
df = p_df[['Project Number','Sector','Date Completed','Expected KWh Annual Production', 'income', 'Purchase Type']]

residential_df = df.loc[df['Sector'] == 'Residential']
residential_df = residential_df.reset_index()
residential_df = residential_df.drop(columns=['index'])
residential_df = residential_df.sort_values('Date Completed', ascending=True)

residential_df.head()

# RENAMING DATAFRAME
r_df = residential_df

# RESIDENTIAL PURCHASE DF
r_purchase_df = r_df.loc[r_df['Purchase Type'] == 'Purchase']

# RESIDENTIAL LEASE DF
r_lease_df = r_df.loc[r_df['Purchase Type'] == 'Lease']

# RESIDENTIAL POWER PURCHASE AGREEMENT DF
r_ppa_df = r_df.loc[r_df['Purchase Type'] == 'Power Purchase Agreement']




# INCENTIVE RATIO/ COST TIME SERIES

In [ ]:
#read in csv data
path = “Resources/project_data_final.csv”
df = pd.read_csv(path)
#drop unnamed columns
df = df.drop(columns=[‘Unnamed: 0’])
#inspect the data
df.head()
#transform date_completed to date_time object
df[‘Date Completed’] = pd.to_datetime(df[‘Date Completed’])
dates = df[‘Date Completed’]

#create a variable to hold the formatted datetime strings
y_m = []

#format datetime as a string with year month
for date in dates:
    y_m.append(date.strftime(‘%Y-%m’))
    
df[‘Year-Month’] = y_m

#inspect data
df.head()

#df.dtypes
#keep only columns in new data frame needed for analysis
analysis_df = df[[‘Year-Month’, ‘Sector’,‘Purchase Type’,  
                 ‘Zip Code’,‘Project Cost’, ‘$Incentive’,
                  ‘income_per_capital’, ‘median_monthly_owner_costs’,
                  ‘Electric Utility’, ‘Total Nameplate kW DC’,
                  ‘$_per_watt_wincentives’, ‘$_per_watt’,
                  ‘Completed Year’, ‘Completed Month’,‘Date Completed’]]

#sort values by ascending year
analysis_df = analysis_df.sort_values(‘Date Completed’, ascending=True).reset_index()

# define function to calculate percent of cost covered by incentives per project
def calculate_incentive_ratio(row):
    if (row[‘Project Cost’] == 0):
         return (“NaN”)
    else:
         return (row[‘$Incentive’]/ row[‘Project Cost’])*100
      
analysis_df[‘% Covered by Incentives’] = analysis_df.apply(calculate_incentive_ratio, axis=1)
# Use apply to save the new column and apply to each row by specifying axis=1
# Create purchase data frame
purchase_data = analysis_df[analysis_df[‘Purchase Type’] == ‘Purchase’]
purchase_data_grouped = purchase_data.groupby([‘Completed Year’]).agg(
                        {
                            ‘Purchase Type’: [‘count’],    
                           ‘% Covered by Incentives’: [‘mean’]
                        }  
                    ).reset_index()


percent_covered_type_purchase = purchase_data_grouped[‘% Covered by Incentives’][‘mean’]
count_type_purchase = purchase_data_grouped[‘Purchase Type’][‘count’]
years_type_purchase = purchase_data_grouped[‘Completed Year’]

# Lease
lease_data = analysis_df[analysis_df[‘Purchase Type’] == ‘Lease’]
lease_data_grouped = lease_data.groupby([‘Completed Year’]).agg(
                        {
                            ‘Purchase Type’: [‘count’],    
                           ‘% Covered by Incentives’: [‘mean’]
                        }  
                    ).reset_index()


percent_covered_type_lease = lease_data_grouped[‘% Covered by Incentives’][‘mean’]
count_type_lease = lease_data_grouped[‘Purchase Type’][‘count’]
years_type_lease = lease_data_grouped[‘Completed Year’]

# Power Purchase Agreement
power_data = analysis_df[analysis_df[‘Purchase Type’] == ‘Power Purchase Agreement’]
power_data_grouped = power_data.groupby([‘Completed Year’]).agg(
                        {
                            ‘Purchase Type’: [‘count’],    
                           ‘% Covered by Incentives’: [‘mean’]
                        }  
                    ).reset_index()


percent_covered_type_power = power_data_grouped[‘% Covered by Incentives’][‘mean’]
count_type_power = power_data_grouped[‘Purchase Type’][‘count’]
years_type_power = power_data_grouped[‘Completed Year’]

# HEATMAP

In [ ]:
# Configure gmaps
gmaps.configure(api_key=apikeys.gmap_key)
%matplotlib inline

#Read in Data
path = os.path.join('..', 'Resources', 'project_data.csv')
solar = pd.read_csv(path)

#Parse lat/long from Location 1 variable
latlong = pd.DataFrame(solar["Location 1"].str.split('(',1).tolist(),
                                   columns = ['junk','realstuff'])
latlong['realstuff'] = latlong.realstuff.str.replace(  ')', '')
latlong = latlong.realstuff.str.split(", ", expand = True )
solar['lat'] = latlong.iloc[:, 0]
solar['long'] = latlong.iloc[:, 1]
solar['lat'] = pd.to_numeric(solar['lat'])
solar['long'] = pd.to_numeric(solar['long'])

del latlong

#04 Produce a dataset summarizing key variables by zip code
solar_byzip_group = solar.groupby(by = ["Zip Code"])
solar_byzip = pd.DataFrame(solar_byzip_group.mean())

#Rename variables
solar_byzip.rename(inplace = True, 
             columns = { 
                 "Project_cost": "Avg_cost", 
                 "$Incentive":"Avg_incentive",
                 "Expected KWh Annual Production": "avg_annualkwh", 
                 "$_per_watt": "Avg_$perwatt",
                 "$_per_watt_wincentives": "Avg_$perwatt_wincentive",
                 "Total Nameplate kW DC": "Avg_installedkW"
             })
                   
#Add in calculated variables
solar_byzip['Total_installedkW'] = solar_byzip_group['Total Nameplate kW DC'].sum()
solar_byzip['Total_AnnualMWh'] = solar_byzip_group['Expected KWh Annual Production'].sum()/1000
solar_byzip['Projects'] = solar_byzip_group['Project Number'].count()
solar_byzip['solar_per_000'] = solar_byzip['Total_installedkW']/(solar_byzip['population']/1000)

solar_byzip['solar_per_000'].describe()
solar_byzip.to_pickle("../Resources/Solar data summary by zip")

check = solar_byzip[[ 'solar_per_000', 'population', 'Total_installedkW']]

# CHOROPLETH

In [ ]:
# =============================================================================
# 2. Format Geo data
# =============================================================================
# Iterate through each station
# Iterate through each row and create polygon
 

geodata = shapefile.Reader("../Resources/zip_code_shapes/cb_2017_us_zcta510_500k.shp")
geos = geodata.shapes()
recs = geodata.records()

lats = []
longs = []

for shape in geos:
    xpoly = []
    ypoly = []
    for point in shape.points:
        lat = point[0]
        long = point[1]
        xpoly.append(lat)
        ypoly.append(long)
    
    lats.append(xpoly)
    longs.append(ypoly)    
    
zipcodes = []    
for rec in recs:
    zipcodes.append(rec[0])    

my_dict = {
        "zipcode": zipcodes
        }
import numpy as np
geodata = pd.DataFrame(my_dict, dtype = np.int64)
geodata['lats'] = lats
geodata['longs'] = longs

geodata.to_json("../Resources/Zip code shape files - Bokeh gmap style.json")

# =============================================================================
#  3. Read in zip level data and merge geometries
# =============================================================================

zipdata = pd.read_pickle("../Resources/Solar data summary by zip")
zipdata.reset_index(level = None, inplace = True)
zipdata.rename(columns = {"Zip Code": "zipcode"}, inplace = True)
zipdata = pd.merge(zipdata, geodata, on = 'zipcode', how = 'inner')
zipdata = zipdata[['zipcode', 'solar_per_000', 'population', 'Total_installedkW', 'lats', 'longs']]
zipdata['solar_rank'] = zipdata.solar_per_000.rank(pct = True)

# =============================================================================
# 4. Produce chart - Penetration per 1000 people
# =============================================================================

#A. Set color scale 
PuBu8.reverse()
color_mapper = LinearColorMapper(palette=PuBu8)

#B. Define source
source = ColumnDataSource(zipdata)

#C. Specify Figure
TOOLS = "pan,wheel_zoom,reset,hover,box_zoom,save"
p = figure(
    title="New York Solar Penetration by zip code", tools=TOOLS,
    x_axis_location=None, y_axis_location=None, 
    plot_width = 1200, plot_height = 800
)


#D. Add Polygons
p.patches('lats', 'longs', source=source,
          fill_color={'field': 'solar_rank', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)


#E. Specify Hover Options
hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Zip Code:", "@zipcode"),
    ("Solar (kW) per 1,000 people", "@solar_per_000{0,0.0}"),
    ("Total Solar Installed (kW)", "@Total_installedkW{0,0}"),
    ("Population", "@population{0,0}"), 
    ("(Long, Lat)", "($x, $y)"),
]


#F. Remove grid lines
p.grid.grid_line_color = None


#G. Add Scale

#H.Show and save figure
show(p)
output_file("../Resources/Zip Code Choropleth Map Bokeh.html")